In [1]:
from flask import Flask, render_template, request
import snscrape.modules.twitter as sntwitter
import pandas as pd
import googletrans
import collections
from googletrans import Translator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import pipeline
#from happytransformer import HappyTextClassification
from textblob import TextBlob
import warnings
warnings.filterwarnings('ignore')
from io import BytesIO
import base64
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%config InlineBackend.figure_format = 'retina'

app = Flask(__name__)


specific_model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")
def sentiment_hf(sentence):
    score = specific_model(sentence)
    return score

model = pipeline(model = "distilbert-base-uncased-finetuned-sst-2-english")
def sentiment_hf2(sentence):
    score = model(sentence)
    return score

def sentiment_textblob(row):
    classifier = TextBlob(row)
    polarity = classifier.sentiment.polarity
    if polarity < 0:
      label = "Negative"
    elif polarity == 0:
      label = "Neutral"
    else:
      label= "Positive"
    return {"polarity":polarity,"label":label}

#happy_tc = HappyTextClassification(model_type="DISTILBERT",  model_name="distilbert-base-uncased-finetuned-sst-2-english")
#def happy(sentence):
    #result = happy_tc.classify_text(sentence)
    #return {"score": result.score , "label": result.label}    


def sentiment_analyzer_scores(sentence):
    analyser = SentimentIntensityAnalyzer()
    score = analyser.polarity_scores(sentence)
    score = (score['compound'])
    if score >= 0.05 :
      return {"score": score , "label": "Positive"}
    elif score <= - 0.05 :
      return {"score": score , "label": "Negative"}
    else :
      return {"score": score , "label": "Neutral"}


@app.route("/")
def home():
    return render_template("usernamereply.html")

def fun(q,Username,Count):
  attributes_container = []
  for i,tweet_reply in enumerate(sntwitter.TwitterSearchScraper(q).get_items()):
    if Count != -1 and len(attributes_container) >= Count:
        break
    if tweet_reply.inReplyToUser != None :
      custUsername = tweet_reply.inReplyToUser.username
      for j,tweet in enumerate(sntwitter.TwitterSearchScraper('@'+Username+ ' from:'+custUsername).get_items()):
        if '@'+Username in str(tweet.rawContent):
          attributes_container.append([tweet.user.username ,tweet.date, tweet.renderedContent , tweet_reply.renderedContent])
          break
 
  tweetsAndReplies = pd.DataFrame(attributes_container, columns=["user" ,"dateCreated","origTweet" ,"userReply" ])
  tweetsAndReplies = tweetsAndReplies.drop_duplicates(keep='first')
  tweetsAndReplies['origTweetTranslated'] = tweetsAndReplies['origTweet'].apply(lambda x: Translator().translate(x, dest='en').text)
  tweetsAndReplies['userReplyTranslated'] = tweetsAndReplies['userReply'].apply(lambda x: Translator().translate(x, dest='en').text)


  tweetsAndReplies['vaderOnOrig'] = tweetsAndReplies['origTweetTranslated'].apply(lambda x: sentiment_analyzer_scores(str(x)))
  tweetsAndReplies['hfOnOrig'] = tweetsAndReplies['origTweetTranslated'].apply(lambda x: sentiment_hf(str(x)))
  tweetsAndReplies['hf2OnOrig'] = tweetsAndReplies['origTweetTranslated'].apply(lambda x: sentiment_hf2(str(x)))
  tweetsAndReplies['textOnOrig'] = tweetsAndReplies['origTweetTranslated'].apply(lambda x: sentiment_textblob(str(x)))
  # tweetsAndReplies['happyOnOrig'] = tweetsAndReplies['origTweetTranslated'].apply(lambda x: happy(str(x)))


  tweetsAndReplies['vaderOnuser'] = tweetsAndReplies['userReplyTranslated'].apply(lambda x: sentiment_analyzer_scores(str(x)))
  tweetsAndReplies['hfOnuser'] = tweetsAndReplies['userReplyTranslated'].apply(lambda x: sentiment_hf(str(x)))
  tweetsAndReplies['hf2Onuser'] = tweetsAndReplies['userReplyTranslated'].apply(lambda x: sentiment_hf2(str(x)))
  tweetsAndReplies['textOnuser'] = tweetsAndReplies['userReplyTranslated'].apply(lambda x: sentiment_textblob(str(x)))
  # tweetsAndReplies['happyOnuser'] = tweetsAndReplies['userReplyTranslated'].apply(lambda x: happy(str(x)))

  tweetsAndReplies["Finaluser"] = 'neg'
  tweetsAndReplies['FinalOrig'] = 'neg'
  for index, row in tweetsAndReplies.iterrows():
    arr = [str(row["vaderOnOrig"]["label"]).lower()[0:3], str(row["hfOnOrig"][0]["label"]).lower()[0:3] , str(row["hf2OnOrig"][0]["label"]).lower()[0:3] , str(row["textOnOrig"]["label"]).lower()[0:3] ]
    count = collections.Counter(arr)
    
    if count['neu']>= max(count['neg'] , count['pos']): 
      tweetsAndReplies.at[index , "FinalOrig"] = 'Neutral'
    elif count['pos'] >= max(count['neu'] , count['neg']):
      tweetsAndReplies.at[index , "FinalOrig"] = 'Positive'
    else:
      tweetsAndReplies.at[index , "FinalOrig"] = 'Negative'
      
    arr = [str(row["vaderOnuser"]["label"]).lower()[0:3], str(row["hfOnuser"][0]["label"]).lower()[0:3] , str(row["hf2Onuser"][0]["label"]).lower()[0:3] , str(row["textOnuser"]["label"]).lower()[0:3] ]
    count = collections.Counter(arr)
    
    if count['neu']>= max(count['neg'] , count['pos']): 
      tweetsAndReplies.at[index , "Finaluser"] = 'Neutral'
    elif count['pos'] >= max(count['neu'] , count['neg']):
      tweetsAndReplies.at[index , "Finaluser"] = 'Positive'
    else:
      tweetsAndReplies.at[index , "Finaluser"] = 'Negative'
  tweetsAndReplies.drop(columns=['vaderOnuser', 'hfOnuser' , 'hf2Onuser' , 'textOnuser' ] , axis=1 , inplace = True)
  tweetsAndReplies.drop(columns=['vaderOnOrig', 'hfOnOrig' , 'hf2OnOrig' , 'textOnOrig' ] , axis=1 , inplace = True)

  return tweetsAndReplies

@app.route('/usernamereply', methods=['POST'])
def res():
  Username = request.form["string-input"]
  Since = request.form["date1-input"]
  Until = request.form["date2-input"]
  q = ""  
  q += f"from:{Username}"
  if Until!='': 
    q+= f" until:{Until}" 
  if Since!='': 
    q += f" since:{Since}"
  Count = 50
  result = fun(q,Username,Count)
  _, axes = plt.subplots(nrows=1, ncols=2 ,figsize=(12, 4))
  sns.countplot(x='FinalOrig', data=result, ax=axes[0])
  sns.countplot(x='Finaluser', data=result, ax=axes[1])
  img = BytesIO()
  plt.savefig(img , format='png')
  plt.close()
  img.seek(0)
  plot_url = base64.b64encode(img.getvalue())

  same = 0 
  lessdev = 0 
  contra = 0
  for index, row in result.iterrows():
    ValUser = str(result._get_value(index , 'Finaluser'))
    ValOrig = str(result._get_value(index , 'FinalOrig'))
    if ValUser==ValOrig:
      same = same + 1
    elif ValUser=='Neutral' or ValOrig=='Neutral':
      lessdev = lessdev+1
    else:
      contra = contra + 1

  my_dict = {'Matching Intent': same, 'Neutral Intent': lessdev, 'Contradicting Intent': contra}
  names = list(my_dict.keys())
  values = list(my_dict.values())
  colors = sns.color_palette('muted')
  plt.bar(range(len(my_dict)), values, tick_label=names , color = colors)

  img2 = BytesIO()
  plt.savefig(img2 , format='png')
  plt.close()
  img2.seek(0)
  pie_url = base64.b64encode(img2.getvalue())
  
  plt.pie(values, labels = names )
  img3 = BytesIO()
  plt.savefig(img3 , format='png')
  plt.close()
  img3.seek(0)
  pie2_url = base64.b64encode(img3.getvalue())

  return render_template('resultkeyreply.html',tables=[result.to_html()], titles=result.columns.values,plot_url = plot_url.decode('utf-8') , pie_url = pie_url.decode('utf-8') , pie2_url = pie2_url.decode('utf-8'))

c:\Users\ANUSKA RAY\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if __name__ == "__main__":
    app.run(port=5001,debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [08/Apr/2023 21:41:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2023 21:44:11] "POST /usernamereply HTTP/1.1" 200 -
